make a copy of this file.
upload cleaned_reddit.csv to the notebook.
make a copy of final_model to your own drive.

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import torch

In [ ]:
reddit = pd.read_csv("cleaned_reddit.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
reddit.shape

(84162, 4)

In [ ]:
reddit.head(5)

,text,created_date,subreddit,engagement_score
0,This is ENTIRELY CURIOSITY. I am NOT currently...,"09/24,25",Banking,0.661168
1,Im am so tired of being understaffed. I work a...,"09/24,25",Banking,0.878203
2,"So, im a college student and i have a student ...","09/24,25",Banking,0.664327
3,Hi Im not sure where to post this but figured ...,"09/24,25",Banking,0.686197
4,"So, this is my case, for background, Im a free...","09/24,25",Banking,0.416601


In [ ]:
# REPLACE WITH YOUR OWN SEGMENT OF DATA
real_data = reddit.head(10)

# Load the trained model and tokenizer
# MAKE SURE YOU SAVED FINAL_MODEL AT THE RIGHT PATH
model_path = '/content/drive/MyDrive/final_model'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# Get texts and clean them
texts = real_data['text'].tolist()
scores = real_data['engagement_score'].tolist()

# Clean the texts - convert to strings and handle None/NaN values
texts = [str(text) if pd.notna(text) else "" for text in texts]

# Filter out empty strings
valid_indices = [i for i, text in enumerate(texts) if text.strip()]
valid_texts = [texts[i] for i in valid_indices]
valid_scores = [scores[i] for i in valid_indices]

print(f"Processing {len(valid_texts)} valid texts out of {len(texts)} total rows")

# Get predictions
inputs = tokenizer(valid_texts, padding=True, truncation=True, max_length=128, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_classes = torch.argmax(predictions, dim=-1)

# Convert predictions to labels
id2label = model.config.id2label
predicted_labels = [id2label[int(pred_id)] for pred_id in predicted_classes.cpu().numpy()]

# Get confidence scores
confidence_scores = [predictions[i][predicted_classes[i]].item() for i in range(len(predicted_classes))]

# Create results for valid texts only
results_df = pd.DataFrame({
    'text': valid_texts,
    'predicted_label': predicted_labels,
    'confidence': confidence_scores,
    'engagement_score': valid_scores
})

# Save to Google Drive
# MAKE SURE TO CHANGE NAME SO YOU DON'T OVERWRITE THE FILE (if you run this cell multiple times)
### for example: '/content/drive/MyDrive/reddit_predictions_batch1.csv' or batch2, etc

results_df.to_csv('/content/drive/MyDrive/reddit_predictions.csv', index=False)

# Display results
print("\nPredictions complete!")
print(f"Total rows: {len(results_df)}")
print("\nFirst 10 predictions:")
print(results_df.head(10))

print("\nCategory distribution:")
print(results_df['predicted_label'].value_counts())

print(f"\nResults saved to: /content/drive/MyDrive/reddit_predictions.csv")

Processing 9 valid texts out of 10 total rows

Predictions complete!
Total rows: 9

First 10 predictions:
                                                text    predicted_label  \
0  This is ENTIRELY CURIOSITY. I am NOT currently...      Notifications   
1  Im am so tired of being understaffed. I work a...  Branch Experience   
2  So, im a college student and i have a student ...   Security & Fraud   
3  Hi Im not sure where to post this but figured ...   Security & Fraud   
4  So, this is my case, for background, Im a free...        ATM Service   
5  With the CapOne changes I am thinking of chang...     Fees & Charges   
6  Today was the third time I receive a call from...   Rewards & Offers   
7  Im about to start interviewing with JP Morgan ...     Online Banking   
8  My bank decided to freeze my checking and cred...     Fees & Charges   

   confidence  engagement_score  
0    0.223475          0.661168  
1    0.585316          0.878203  
2    0.279714          0.664327  
3    0.